In [46]:
import os
import numpy as np
import qiskit
import math
import matplotlib.pyplot as plt

from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService
from dotenv import load_dotenv

load_dotenv()
IBM_API_TOKEN = os.getenv("IBM_API_TOKEN")

In [24]:
N = int(input("Ingrese un número que se pueda descomponer en 2 factores primos: "))
lim_inf = 2
lim_sup = N - 2
qubit_number = int(np.ceil(np.log2(N + 1))) if N > 0 else 1

In [ ]:
def QFT(N: int):
    """Transformada de Fourier Cuántica para N qubits."""
    circuit = qiskit.QuantumCircuit(N)
    for i in range(N-1, -1, -1):
        circuit.h(i)
        for j in range(i-1, -1, -1):
            theta = math.pi / (2 ** (i - j))
            circuit.cp(theta, j, i)
    
    for i in range(N // 2):
        circuit.swap(i, N - i - 1)

    compuert = circuit.to_gate()
    compuert.name = "QFT"
    return compuert


In [34]:
circuit = qiskit.QuantumCircuit(4)
circuit.h(range(4))
circuit.append(QFT(4), range(4))
circuit.draw()

┌───┐┌──────┐
q_0: ┤ H ├┤0     ├
     ├───┤│      │
q_1: ┤ H ├┤1     ├
     ├───┤│  QFT │
q_2: ┤ H ├┤2     ├
     ├───┤│      │
q_3: ┤ H ├┤3     ├
     └───┘└──────┘

In [ ]:
# service = QiskitRuntimeService(channel="ibm_quantum", token=IBM_API_TOKEN)
# backends = service.backends()
# for backend in backends:
#     print(f"- {backend.name}")

/tmp/ipykernel_2831/3788642992.py:1: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum", token=IBM_API_TOKEN)


- ibm_brisbane
- ibm_sherbrooke
